In [ ]:
import cv2
import time
import numpy as np
from matplotlib import pyplot as plt
from google.colab.patches import cv2_imshow

frameFiles = ['S1.jpg', 'S2.jpg', 'S3.jpg', 'S5.jpg', 'S6.jpg']

colorFrames = [cv2.imread(i) for i in frameFiles]
# colorFrames = [cv2.rotate(cv2.imread(i), cv2.cv2.ROTATE_90_CLOCKWISE) for i in frameFiles]
grayFrames = [cv2.cvtColor(i, cv2.COLOR_BGR2GRAY) for i in colorFrames]


In [ ]:
# cv2_imshow(colorFrames[3])
# print(len(colorFrames))

In [ ]:
orb_detector = cv2.ORB_create(5000)

kps = []
fds = []
for i in grayFrames:
    kp, fd = orb_detector.detectAndCompute(i, None)
    kps.append(kp)
    fds.append(fd)

# for i in kps:
#     print(len(i))
# print(len(kps[1]))
# print(len(fds))

In [ ]:
class matchTrip:
    def __init__(self, row=-1, col=-1, matchCt=0):
        self.row = row
        self.col = col
        self.matchCt = matchCt

    def __lt__(self, other):
        return self.matchCt < other.matchCt

    def show(self):
        return "Row " + str(self.row) + " Col " + str(self.col) + " Matches " + str(self.matchCt)


In [ ]:
matchTable = np.zeros((len(fds), len(fds)))
matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck = True)

matchesTable = []
hMatrix = []
tripletList = []

for i in range(len(fds)):
    rmatches = []
    rMatrix = []
    for j in range(i+1, len(fds)):
        matches = matcher.match(fd[i], fd[j])
        matches.sort(key = lambda x: x.distance)
        matchTable[i][j] = matchTable[j][i] = len(matches)*90
        rmatches.append(matches[:len(matches)*90])
        tripletList.append(matchTrip(i, j, len(matches)*90))
        dst_pts = np.float32([kps[i][m.queryIdx].pt for m in matches]).reshape(-1,1,2)
        src_pts = np.float32([kps[j][m.trainIdx].pt for m in matches]).reshape(-1,1,2)
        M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
        rMatrix.append(M)
    matchesTable.append(rmatches)
    hMatrix.append(rMatrix)


In [ ]:
print(matchTable)
tripletList.sort()
for m in tripletList:
    print(m.show())
# print(tripletList)

[[   0. 1080. 1080. 1260. 1260.]
 [1080.    0. 1440. 1350. 1260.]
 [1080. 1440.    0. 1260. 1080.]
 [1260. 1350. 1260.    0. 1440.]
 [1260. 1260. 1080. 1440.    0.]]
Row 0 Col 1 Matches 1080
Row 0 Col 2 Matches 1080
Row 2 Col 4 Matches 1080
Row 0 Col 3 Matches 1260
Row 0 Col 4 Matches 1260
Row 1 Col 4 Matches 1260
Row 2 Col 3 Matches 1260
Row 1 Col 3 Matches 1350
Row 1 Col 2 Matches 1440
Row 3 Col 4 Matches 1440


In [ ]:
hPair = []
for i in range(len(tripletList)-1, len(tripletList)-len(fds)+1, -1):
    hPair.append([tripletList[i].row, tripletList[i].col])
print(hPair)

[[3, 4], [1, 2], [1, 3]]


In [ ]:
matchSum = np.zeros((len(fds), 1))
maxIdx = -1
maxMatches = -1
for i in range(len(fds)):
    for j in range(len(fds)):
        matchSum[i] += matchTable[i][j]
    if matchSum[i] > maxMatches:
        maxMatches = matchSum[i]
        maxIdx = i

print(matchSum)
print("Index of maximum matched Frame ", maxIdx)

[[4680.]
 [5130.]
 [4860.]
 [5310.]
 [5040.]]
Index of maximum matched Frame  3


In [ ]:
class pair:
    def __init__(self, id, m):
        self.idx = id
        self.matches = m

    def __lt__(self, other):
        return self.matches < other.matches
        
class PriorityQueue(object):
    def __init__(self):
        self.queue = []
  
    def __str__(self):
        return ' '.join([str(i) for i in self.queue])
  
    # for checking if the queue is empty
    def isEmpty(self):
        return len(self.queue) == 0
  
    # for inserting an element in the queue
    def insert(self, data):
        self.queue.append(data)
  
    # for popping an element based on Priority
    def delete(self):
        try:
            max = 0
            for i in range(len(self.queue)):
                if self.queue[i] > self.queue[max]:
                    max = i
            item = self.queue[max]
            del self.queue[max]
            return item
        except IndexError:
            print()
            exit()

In [ ]:
frameOrder = []
vis = []
q = PriorityQueue()
q.insert(matchTrip(-1, maxIdx, maxMatches))
count = 0
while count!=len(fds):
    tp = q.delete()
    k = tp.col
    if k not in vis:
        frameOrder.append([tp.row, tp.col])
        count += 1
        vis.append(k)
        for j in range(len(fds)):
            q.insert(matchTrip(k, j, matchTable[k][j]))


print(frameOrder)

[[-1, 3], [3, 4], [3, 1], [1, 2], [3, 0]]


In [ ]:
def warpTwoImages(img1, img2, H):
    h1,w1 = img1.shape[:2]
    h2,w2 = img2.shape[:2]
    # print("bpts -> ", bpts)
    # bpts = np.float32(bpts).reshape(-1, 1, 2)
    pts1 = np.float32([[0,0],[0,h1],[w1,h1],[w1,0]]).reshape(-1,1,2)
    pts2 = np.float32([[0,0],[0,h2],[w2,h2],[w2,0]]).reshape(-1,1,2)
    pts2_ = cv2.perspectiveTransform(pts2, H)
    pts = np.concatenate((pts1, pts2_), axis=0)
    # bpts = np.concatenate((bpts, pts2_), axis=0)
    [xmin, ymin] = np.int32(pts.min(axis=0).ravel() - 0.5)
    [xmax, ymax] = np.int32(pts.max(axis=0).ravel() + 0.5)
    t = [-xmin,-ymin]
    # print(pts)
    Ht = np.array([[1,0,t[0]],[0,1,t[1]],[0,0,1]]) # translate

    result = cv2.warpPerspective(img2, Ht.dot(H), (xmax-xmin, ymax-ymin))
    # print(img2)
    # im2_boundary = cv2.warpPerspective([[0,0],[0,h2],[w2,h2],[w2,0]], Ht.dot(H), (xmax-xmin, ymax-ymin))
    # print(im2_boundary)
    result[t[1]:h1+t[1],t[0]:w1+t[0]] = img1
    return result, cv2.perspectiveTransform(pts2_, Ht)



In [ ]:
h1,w1 = colorFrames[frameOrder[0]].shape[:2]
x = np.array([[0,0],[0,h1],[w1,h1],[w1,0]]).reshape(-1,1,2)
print(x)

TypeError: ignored

In [ ]:
hPair = [[3, 4], [3, 2], [2, 1]]

In [ ]:
riIdx = maxIdx

In [ ]:
h1,w1 = colorFrames[3].shape[:2]
x = np.array([[0,0],[0,h1],[w1,h1],[w1,0]]).reshape(-1,1,2)
rI = colorFrames[riIdx]
boundaryPts = x
hTimes = []
warpTimes = []
hm = []
for p in hPair:
    i = p[0]
    j = p[1]
    
    dst_pts = np.float32([kps[i][m.queryIdx].pt for m in matches]).reshape(-1,1,2)
    src_pts = np.float32([kps[j][m.trainIdx].pt for m in matches]).reshape(-1,1,2)

    # st = time.time()
    M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 6.0)
    hm.append(M)
    # et = time.time()
    # hTimes.append(et-st)
    # st = time.time()
    rI, bps = warpTwoImages(colorFrames[i], colorFrames[j], M)
    boundaryPts = np.concatenate((boundaryPts, bps), axis=0)
    # et = time.time()
    # warpTimes.append(et-st)
    # boundaryPts.append(pts)

# print(hTimes)
# print(warpTimes)

In [ ]:
print(len(hm))

In [ ]:
dst_pts = np.float32([kps[i][m.queryIdx].pt for m in matches]).reshape(-1,1,2)
src_pts = np.float32([kps[j][m.trainIdx].pt for m in matches]).reshape(-1,1,2)
rI, bps = warpTwoImages(colorFrames[3], colorFrames[1], hm[1]@hm[2])
boundaryPts = np.concatenate((boundaryPts, bps), axis=0)

In [ ]:
# rI = colorFrames[frameOrder[0]]
# boundaryPts = x
# hTimes = []
# warpTimes = []
# for i in frameOrder[1:]:  
#     grI = cv2.cvtColor(rI, cv2.COLOR_BGR2GRAY)
#     rIkp, rIfd = orb_detector.detectAndCompute(grI, None)
#     matches = matcher.match(rIfd, fds[i])
#     matches.sort(key = lambda x: x.distance)
#     matches = matches[:int(len(matches)*90)]

#     # gkp = [rIkp[m.queryIdx].pt for m in matches]
#     # correct_matched_kp = [good_kp[i] for i in range(len(good_kp)) if mask[i]]

#     dst_pts = np.float32([rIkp[m.queryIdx].pt for m in matches]).reshape(-1,1,2)
#     src_pts = np.float32([kps[i][m.trainIdx].pt for m in matches]).reshape(-1,1,2)

    
#     st = time.time()
#     M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 6.0)
#     et = time.time()
#     hTimes.append(et-st)
#     st = time.time()
#     rI, boundaryPts = warpTwoImages(rI, colorFrames[i], M, boundaryPts)
#     et = time.time()
#     warpTimes.append(et-st)
#     # boundaryPts.append(pts)

# print(hTimes)
# print(warpTimes)

In [ ]:
cv2_imshow(rI)

In [ ]:
print(boundaryPts)
pts = boundaryPts

In [ ]:
pts = np.array([[pts[:]]], np.int32)
pts = pts.reshape((-1,1,2))

print(pts)
rI = np.zeros((1000, 1000,3), dtype = "uint8")
for i in range(0, len(pts), 4):
    cv2.polylines(rI,[pts[i:i+4]],True,(0,225-25*int(i/4),225-25*int(i/4)),3)  
# img1_color = cv2.resize(img1_color, (1000,650))
# cv2.polylines(rI,[pts[:4]],True,(0,255,255),3)
# cv2.polylines(rI,[pts[4:]],True,(255,255,),3)
cv2_imshow(rI)

In [ ]:
# dst_pts = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1,1,2)
# src_pts = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1,1,2)
# M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

# result = warpTwoImages(img1_color, img2_color, M)